# Datasets with PyTorch


In [10]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as scikit_learn
%matplotlib inline

In [44]:
from sklearn.datasets import load_iris
df = load_iris()

In [46]:
df = pd.DataFrame(data= np.c_[df['data'], df['target']],
                     columns= df['feature_names'] + ['target'])

In [52]:
X = df.drop('target',axis=1).values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
# y_train = F.one_hot(torch.LongTensor(y_train))  # not needed with Cross Entropy Loss
# y_test = F.one_hot(torch.LongTensor(y_test))
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
class Model(nn.Module):
    # 4 features, 8 neurons in first hidden layer, 9 neurons in 2nd layer, 3 output features 
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        # how many layers?
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        # activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [41]:
# Instantiate the Model class using parameter defaults:
torch.manual_seed(32)
model = Model()

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Train the model

In [58]:
epochs = 100
losses = []

for i in range(epochs):
    i+=1
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  1  loss: 1.15074503
epoch: 11  loss: 0.93714488
epoch: 21  loss: 0.77962416
epoch: 31  loss: 0.60785323
epoch: 41  loss: 0.39894527
epoch: 51  loss: 0.25249198
epoch: 61  loss: 0.14927688
epoch: 71  loss: 0.10029560
epoch: 81  loss: 0.08100693
epoch: 91  loss: 0.07216033


In [63]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    y_val = model.forward(X_test)
    loss = criterion(y_val, y_test)
print(f'{loss:.8f}')

0.05816953


In [65]:
correct = 0
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = model.forward(data)
        print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-2.1234,  4.8067, -0.8803])     1
 2. tensor([-1.7920,  5.3100, -1.5693])     1
 3. tensor([  6.3723,   0.8741, -10.0971])  0
 4. tensor([-3.9129,  4.5951,  1.1509])     1
 5. tensor([-7.4882,  3.1953,  5.7839])     2
 6. tensor([-10.5202,   1.6381,   9.6291])  2
 7. tensor([  6.3364,   1.0237, -10.1951])  0
 8. tensor([  7.0690,   0.7370, -10.9620])  0
 9. tensor([-7.2218,  3.3422,  5.3528])     2
10. tensor([-9.4170,  2.5675,  8.1028])     2
11. tensor([-9.9029,  2.3388,  8.7141])     2
12. tensor([ 6.2942,  0.6938, -9.8046])     0
13. tensor([-9.3335,  2.1817,  8.1917])     2
14. tensor([-3.7832,  4.5046,  1.0603])     1
15. tensor([-7.8793,  3.0060,  6.2225])     2
16. tensor([-1.8810,  5.1571, -1.3572])     1
17. tensor([-5.7107,  3.5003,  3.6612])     2
18. tensor([  7.2014,   0.7687, -11.1842])  0
19. tensor([-3.2961,  4.7939,  0.3307])     1
20. tensor([-7.7822,  3.7560,  5.7040])     2
21. tensor([  6.6703,   0.8191, -10.4707])  0
22. tensor([  7.4580,   0.9259, -1

In [73]:
torch.save(model, 'IrisDatasetModel.pt')